<a href="https://colab.research.google.com/github/Ravikumara666/Ravikumara666/blob/main/Adobe_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
import pickle
import os

def load_csv_pairs(input_csv_paths, output_csv_paths):
    input_data = []
    output_data = []

    for input_csv, output_csv in zip(input_csv_paths, output_csv_paths):
        input_df = pd.read_csv(input_csv, header=None)
        output_df = pd.read_csv(output_csv, header=None)

        # Assuming each CSV contains polylines and their corresponding shapes
        input_data.append(input_df.values)
        output_data.append(output_df.values)

    return input_data, output_data

def flatten_and_pad(data, fixed_size):
    flattened_data = []
    for shape_list in data:
        sample_data = []
        for shape in shape_list:
            flattened_shape = shape.flatten()
            if len(flattened_shape) > fixed_size:
                flattened_shape = flattened_shape[:fixed_size]
            elif len(flattened_shape) < fixed_size:
                flattened_shape = np.pad(flattened_shape, (0, fixed_size - len(flattened_shape)), mode='constant', constant_values=0)
            sample_data.append(flattened_shape)
        flattened_data.append(np.concatenate(sample_data))

    return flattened_data

def preprocess_data(input_data, output_data, fixed_size=1000000):
    standardized_input_data = flatten_and_pad(input_data, fixed_size)
    standardized_output_data = flatten_and_pad(output_data, fixed_size)

    # Convert lists to NumPy arrays
    X = np.array(standardized_input_data)
    y = np.array(standardized_output_data)

    # Ensure that the shapes are consistent
    assert X.shape[0] == y.shape[0], "Mismatch in number of samples"
    assert X.shape[1] == y.shape[1], "Mismatch in input data shape"
    assert y.shape[1] == X.shape[1], "Mismatch in output data shape"

    return X, y

def save_predictions(predictions, file_path):
    # Convert predictions to a DataFrame and save to CSV
    pd.DataFrame(predictions).to_csv(file_path, header=False, index=False)

def visualize_results(X_test, y_test, y_pred, num_samples=5, output_png_dir='output_png'):
    if not os.path.exists(output_png_dir):
        os.makedirs(output_png_dir)

    for i in range(min(num_samples, len(X_test))):
        plt.figure(figsize=(12, 4))

        # Reshape to original form for visualization if necessary
        X_test_shape = X_test[i].reshape(-1, 2)  # Adjust if needed
        y_test_shape = y_test[i].reshape(-1, 2)  # Adjust if needed
        y_pred_shape = y_pred[i].reshape(-1, 2)  # Adjust if needed

        plt.subplot(1, 3, 1)
        plt.title('Irregular Shape')
        plt.scatter(X_test_shape[:, 0], X_test_shape[:, 1], s=1)

        plt.subplot(1, 3, 2)
        plt.title('Corrected Shape')
        plt.scatter(y_test_shape[:, 0], y_test_shape[:, 1], s=1)

        plt.subplot(1, 3, 3)
        plt.title('Predicted Shape')
        plt.scatter(y_pred_shape[:, 0], y_pred_shape[:, 1], s=1)

        plt.savefig(os.path.join(output_png_dir, f'result_{i}.png'))
        plt.close()

def predict_and_save_new_data(model, scaler_X, scaler_y, new_input_csv_path, output_png_dir='output_png', fixed_size=1000000):
    # Load and preprocess new data
    new_input_data = pd.read_csv(new_input_csv_path, header=None).values
    preprocessed_input = flatten_and_pad([new_input_data], fixed_size)[0]

    # Standardize the new data
    scaled_input = scaler_X.transform([preprocessed_input])

    # Predict
    scaled_predictions = model.predict(scaled_input)

    # Inverse transform to original scale
    predictions = scaler_y.inverse_transform(scaled_predictions)

    # Reshape predictions for visualization
    # Assuming predictions need to be reshaped back into original 2D shape
    num_points = len(predictions[0]) // 2
    reshaped_predictions = predictions[0].reshape(num_points, 2)

    # Save as PNG
    plt.figure(figsize=(8, 8))
    plt.scatter(reshaped_predictions[:, 0], reshaped_predictions[:, 1], s=1)
    plt.title('Predicted Shape')
    plt.savefig(os.path.join(output_png_dir, 'new_data_predictions.png'))
    plt.close()

def main():
    # Example CSV file paths for training
    input_csv_files = [
        '/content/frag1.csv',
        '/content/frag2.csv',
        '/content/occlusion1.csv',
        '/content/isolated.csv'
    ]

    output_csv_files = [
        '/content/frag01_sol.csv',
        '/content/frag2_sol.csv',
        '/content/occlusion1_sol.csv',
        '/content/isolated_sol.csv'
    ]

    input_data, output_data = load_csv_pairs(input_csv_files, output_csv_files)
    X, y = preprocess_data(input_data, output_data)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Standardize features
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train = scaler_X.fit_transform(X_train)
    X_test = scaler_X.transform(X_test)
    y_train = scaler_y.fit_transform(y_train)
    y_test = scaler_y.transform(y_test)

    # Train a Multi-layer Perceptron Regressor (MLP)
    model = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=200, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions and evaluate
    y_pred = model.predict(X_test)
    print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")

    # Save the trained model
    with open('trained_model.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)
    with open('scaler_X.pkl', 'wb') as scaler_X_file:
        pickle.dump(scaler_X, scaler_X_file)
    with open('scaler_y.pkl', 'wb') as scaler_y_file:
        pickle.dump(scaler_y, scaler_y_file)

    # Visualize results
    visualize_results(X_test, y_test, y_pred)

    # Predict and save results for new data
    new_input_csv_path = '/content/frag0.csv'
    predict_and_save_new_data(model, scaler_X, scaler_y, new_input_csv_path)

if __name__ == '__main__':
    main()
